# Elevation Data Access and Processing
## Accessing Public Digital Elevation Models for EEMT Analysis

**Learning Objectives:**
- Access elevation data from multiple public sources (3DEP, OpenTopography, Global DEMs)
- Understand different DEM types, resolutions, and coordinate systems
- Validate and quality-control elevation datasets
- Process DEMs for EEMT calculations
- Visualize topographic data effectively

**Prerequisites:**
- Basic understanding of GIS concepts
- Familiarity with coordinate reference systems
- Python geospatial libraries (rasterio, geopandas)

**Estimated Time:** 40 minutes

## 1. Environment Setup

In [ ]:
# Environment and package setup
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Add utilities to path
sys.path.append('../utilities')

# Core scientific computing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Geospatial libraries
import rasterio
import rasterio.plot
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
import contextily as ctx
import folium
from folium import plugins

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider, Dropdown, IntSlider

# Custom utilities
from data_access import (
    download_sample_dem, download_opentopo_dem, validate_dataset, 
    print_validation_report, ensure_output_directory
)

# Configure display
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

print("✅ Environment setup complete")
print(f"📂 Working directory: {os.getcwd()}")

## 2. Overview of Elevation Data Sources

### 2.1 Available Data Sources

In [ ]:
# Create overview table of elevation data sources
elevation_sources = [
    {
        'Source': 'USGS 3DEP',
        'Resolution': '1m, 10m, 30m',
        'Coverage': 'United States',
        'API': 'Yes',
        'Best_For': 'High-resolution US analysis',
        'Data_Type': 'Lidar-derived DSM/DTM',
        'Vertical_Accuracy': '±0.1-0.5m',
        'Format': 'Cloud-Optimized GeoTIFF'
    },
    {
        'Source': 'OpenTopography SRTM GL1',
        'Resolution': '30m (~1 arc-second)',
        'Coverage': 'Global (60°N-56°S)',
        'API': 'Yes',
        'Best_For': 'Global analysis',
        'Data_Type': 'Radar interferometry',
        'Vertical_Accuracy': '±16m',
        'Format': 'GeoTIFF'
    },
    {
        'Source': 'OpenTopography ALOS',
        'Resolution': '30m',
        'Coverage': 'Global',
        'API': 'Yes', 
        'Best_For': 'Global high-quality',
        'Data_Type': 'Optical stereo',
        'Vertical_Accuracy': '±5m',
        'Format': 'GeoTIFF'
    },
    {
        'Source': 'Copernicus DEM',
        'Resolution': '30m, 90m',
        'Coverage': 'Global',
        'API': 'Via Copernicus Data Space',
        'Best_For': 'Recent global data',
        'Data_Type': 'TanDEM-X radar',
        'Vertical_Accuracy': '±4m',
        'Format': 'GeoTIFF'
    },
    {
        'Source': 'FABDEM',
        'Resolution': '30m', 
        'Coverage': 'Global',
        'API': 'Via Google Earth Engine',
        'Best_For': 'Forest-corrected terrain',
        'Data_Type': 'Forest-removed SRTM',
        'Vertical_Accuracy': '±2m (forest areas)',
        'Format': 'GeoTIFF'
    }
]

# Convert to DataFrame and display
sources_df = pd.DataFrame(elevation_sources)

# Create interactive table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(sources_df.columns),
        fill_color='lightblue',
        align='left',
        font=dict(size=12, color='black')
    ),
    cells=dict(
        values=[sources_df[col] for col in sources_df.columns],
        fill_color='white',
        align='left',
        font=dict(size=11)
    )
)])

fig.update_layout(
    title='Public Elevation Data Sources for EEMT Analysis',
    height=400
)

fig.show()

print("📊 Key Selection Criteria:")
print("   🏔️  Resolution: Higher resolution for detailed topographic analysis")
print("   🌍 Coverage: Global vs regional availability")
print("   🎯 Accuracy: Vertical accuracy for your application requirements")
print("   🔄 API Access: Programmatic download capabilities")
print("   💾 Format: Cloud-optimized formats for efficient processing")

## 3. Interactive Study Area Selection

### 3.1 Geographic Area Selector

In [ ]:
# Define some example study areas
study_areas = {
    'Arizona Sky Islands': {
        'bbox': [-110.5, 32.0, -110.0, 32.5],
        'description': 'Diverse topography with elevation range 800-2800m',
        'recommended_source': 'USGS 3DEP',
        'resolution': '10m'
    },
    'Colorado Front Range': {
        'bbox': [-105.5, 40.0, -105.0, 40.5],
        'description': 'Mountain terrain with alpine-desert gradient',
        'recommended_source': 'USGS 3DEP', 
        'resolution': '10m'
    },
    'Italian Alps': {
        'bbox': [10.5, 46.0, 11.0, 46.5],
        'description': 'High-relief European mountain terrain',
        'recommended_source': 'Copernicus DEM',
        'resolution': '30m'
    },
    'Australian Outback': {
        'bbox': [133.0, -24.0, 133.5, -23.5],
        'description': 'Arid landscape with low relief',
        'recommended_source': 'SRTM GL1',
        'resolution': '30m'
    },
    'Custom Area': {
        'bbox': [-111.0, 32.0, -110.5, 32.5],
        'description': 'User-defined study area',
        'recommended_source': 'SRTM GL1',
        'resolution': '30m'
    }
}

def create_study_area_map(area_name):
    """Create interactive map showing study area"""
    area_info = study_areas[area_name]
    bbox = area_info['bbox']
    west, south, east, north = bbox
    
    # Calculate center point
    center_lat = (south + north) / 2
    center_lon = (west + east) / 2
    
    # Create folium map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,
        tiles='OpenStreetMap'
    )
    
    # Add study area rectangle
    folium.Rectangle(
        bounds=[[south, west], [north, east]],
        color='red',
        fill=True,
        fillOpacity=0.2,
        popup=f'{area_name}<br>{area_info["description"]}'
    ).add_to(m)
    
    # Add terrain basemap option
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='ESRI World Imagery',
        name='Satellite',
    ).add_to(m)
    
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Terrain_Base/MapServer/tile/{z}/{y}/{x}',
        attr='ESRI Terrain',
        name='Terrain'
    ).add_to(m)
    
    folium.LayerControl().add_to(m)
    
    return m

@interact(
    study_area=Dropdown(
        options=list(study_areas.keys()),
        value='Arizona Sky Islands',
        description='Study Area:'
    )
)
def select_study_area(study_area):
    area_info = study_areas[study_area]
    bbox = area_info['bbox']
    
    print(f"📍 Selected Study Area: {study_area}")
    print(f"📦 Bounding Box: {bbox}")
    print(f"📋 Description: {area_info['description']}")
    print(f"💡 Recommended Source: {area_info['recommended_source']}")
    print(f"🔍 Recommended Resolution: {area_info['resolution']}")
    
    # Calculate area dimensions
    west, south, east, north = bbox
    width_deg = east - west
    height_deg = north - south
    
    # Approximate area in km² (rough calculation)
    lat_avg = (south + north) / 2
    km_per_deg_lat = 111.0
    km_per_deg_lon = 111.0 * np.cos(np.deg2rad(lat_avg))
    
    area_km2 = width_deg * km_per_deg_lon * height_deg * km_per_deg_lat
    
    print(f"📐 Area: ~{area_km2:.0f} km² ({width_deg:.3f}° × {height_deg:.3f}°)")
    
    # Display map
    map_widget = create_study_area_map(study_area)
    display(map_widget)
    
    return bbox

## 4. Data Download and Access

### 4.1 Sample DEM Creation and OpenTopography Access

In [ ]:
# Download sample DEM for immediate use
print("🔄 Creating sample DEM for immediate analysis...")
sample_dem_path = download_sample_dem("../data/elevation")

# Validate the sample DEM
print("\n📊 Sample DEM Validation:")
validation = validate_dataset(sample_dem_path, 'elevation')
print_validation_report(validation)

# Load and display sample DEM
with rasterio.open(sample_dem_path) as src:
    sample_elevation = src.read(1)
    sample_transform = src.transform
    sample_crs = src.crs
    
    # Get extent
    bounds = src.bounds
    extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Elevation map
im1 = axes[0].imshow(sample_elevation, extent=extent, cmap='terrain', origin='upper')
axes[0].set_title('Sample DEM - Elevation')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
plt.colorbar(im1, ax=axes[0], label='Elevation (m)')

# Elevation histogram
valid_elevation = sample_elevation[~np.isnan(sample_elevation)]
axes[1].hist(valid_elevation, bins=50, alpha=0.7, color='brown', edgecolor='black')
axes[1].set_xlabel('Elevation (m)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Elevation Distribution')
axes[1].grid(True, alpha=0.3)

# Add statistics text
stats_text = f'Min: {np.nanmin(valid_elevation):.0f} m\nMax: {np.nanmax(valid_elevation):.0f} m\nMean: {np.nanmean(valid_elevation):.0f} m\nStd: {np.nanstd(valid_elevation):.0f} m'
axes[1].text(0.05, 0.95, stats_text, transform=axes[1].transAxes, 
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n✅ Sample DEM loaded successfully from: {sample_dem_path}")

### 4.2 OpenTopography Data Download

In [ ]:
def download_real_dem_data():
    """
    Download real DEM data from OpenTopography
    """
    # Define a small test area (Arizona)
    test_bbox = [-110.2, 32.2, -110.0, 32.4]  # Small area for quick download
    
    print("🌍 Downloading real elevation data from OpenTopography...")
    print(f"📦 Bounding box: {test_bbox}")
    print("⏱️  This may take 30-60 seconds...")
    
    try:
        # Download SRTM data
        srtm_path = download_opentopo_dem(
            bbox=test_bbox,
            dem_type='SRTMGL1',
            output_dir='../data/elevation'
        )
        
        # Validate downloaded data
        validation = validate_dataset(srtm_path, 'elevation')
        print("\n📊 Downloaded DEM Validation:")
        print_validation_report(validation)
        
        return srtm_path
        
    except Exception as e:
        print(f"❌ Download failed: {e}")
        print("💡 Using sample DEM instead for analysis")
        return sample_dem_path

# Interactive download option
@interact(
    download_real=widgets.ToggleButton(
        value=False,
        description='Download Real DEM',
        button_style='info',
        tooltip='Download actual SRTM data (requires internet)'
    )
)
def dem_download_option(download_real):
    if download_real:
        return download_real_dem_data()
    else:
        print("📦 Using sample DEM for analysis")
        print("💡 Toggle above to download real data from OpenTopography")
        return sample_dem_path

## 5. DEM Processing and Analysis

### 5.1 Topographic Derivatives

In [ ]:
def calculate_topographic_derivatives(elevation_array, pixel_size=30):
    """
    Calculate slope, aspect, and other topographic derivatives
    
    Parameters:
    elevation_array: 2D numpy array of elevation values
    pixel_size: Pixel size in meters
    
    Returns:
    Dictionary with slope, aspect, hillshade, and other derivatives
    """
    # Calculate gradients
    dy, dx = np.gradient(elevation_array, pixel_size)
    
    # Slope in degrees
    slope_rad = np.arctan(np.sqrt(dx**2 + dy**2))
    slope_deg = np.rad2deg(slope_rad)
    
    # Aspect in degrees (0=N, 90=E, 180=S, 270=W)
    aspect_rad = np.arctan2(-dx, dy)
    aspect_deg = np.rad2deg(aspect_rad)
    aspect_deg = (450 - aspect_deg) % 360  # Convert to standard geographic convention
    
    # Hillshade (simple illumination model)
    # Sun elevation = 45°, azimuth = 315° (NW)
    sun_elevation = np.deg2rad(45)
    sun_azimuth = np.deg2rad(315)
    
    hillshade = (np.sin(sun_elevation) * np.cos(slope_rad) +
                np.cos(sun_elevation) * np.sin(slope_rad) *
                np.cos(sun_azimuth - aspect_rad))
    hillshade = np.clip(hillshade, 0, 1) * 255
    
    # Curvature (second derivatives)
    dxx = np.gradient(dx, pixel_size, axis=1)
    dyy = np.gradient(dy, pixel_size, axis=0) 
    dxy = np.gradient(dx, pixel_size, axis=0)
    
    # Plan curvature (curvature perpendicular to slope direction)
    plan_curvature = (dxx * dy**2 - 2 * dxy * dx * dy + dyy * dx**2) / (dx**2 + dy**2 + 1e-10)**1.5
    
    # Profile curvature (curvature in slope direction)
    profile_curvature = (dxx * dx**2 + 2 * dxy * dx * dy + dyy * dy**2) / (dx**2 + dy**2 + 1e-10)**1.5
    
    return {
        'slope': slope_deg,
        'aspect': aspect_deg,
        'hillshade': hillshade,
        'plan_curvature': plan_curvature,
        'profile_curvature': profile_curvature,
        'dx': dx,
        'dy': dy
    }

# Calculate derivatives for sample DEM
print("🔢 Calculating topographic derivatives...")
derivatives = calculate_topographic_derivatives(sample_elevation, pixel_size=100)  # Approximate pixel size

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

# Plot each derivative
plots = [
    ('Elevation', sample_elevation, 'terrain'),
    ('Slope', derivatives['slope'], 'Reds'), 
    ('Aspect', derivatives['aspect'], 'hsv'),
    ('Hillshade', derivatives['hillshade'], 'gray'),
    ('Plan Curvature', derivatives['plan_curvature'], 'RdBu'),
    ('Profile Curvature', derivatives['profile_curvature'], 'RdBu')
]

for i, (title, data, cmap) in enumerate(plots):
    im = axes[i].imshow(data, extent=extent, cmap=cmap, origin='upper')
    axes[i].set_title(title)
    axes[i].set_xlabel('Longitude')
    axes[i].set_ylabel('Latitude')
    
    # Add colorbar
    plt.colorbar(im, ax=axes[i], shrink=0.8)
    
    # Add statistics
    valid_data = data[~np.isnan(data)]
    if len(valid_data) > 0:
        stats_text = f'Range: {np.nanmin(valid_data):.1f} to {np.nanmax(valid_data):.1f}'
        axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes,
                    verticalalignment='top', fontsize=9,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

# Print summary statistics
print("\n📊 Topographic Derivative Statistics:")
for name, data in derivatives.items():
    if name not in ['dx', 'dy']:  # Skip raw gradients
        valid_data = data[~np.isnan(data)]
        if len(valid_data) > 0:
            print(f"   {name:15}: {np.nanmin(valid_data):8.2f} to {np.nanmax(valid_data):8.2f} (mean: {np.nanmean(valid_data):6.2f})")

print("\n💡 Interpretation:")
print("   🏔️  Slope: Steepness of terrain (0° = flat, 90° = vertical)")
print("   🧭 Aspect: Direction of steepest descent (0° = North, 180° = South)")
print("   ☀️  Hillshade: Illumination simulation for visualization")
print("   📐 Plan Curvature: Convergence/divergence of flow (+ = ridges, - = valleys)")
print("   📈 Profile Curvature: Acceleration/deceleration of flow (+ = convex, - = concave)")

### 5.2 Interactive Topographic Analysis

In [ ]:
@interact(
    analysis_type=Dropdown(
        options=['Slope Distribution', 'Aspect Analysis', 'Elevation Profiles', 'Terrain Classification'],
        value='Slope Distribution',
        description='Analysis:'
    ),
    slope_threshold=FloatSlider(
        min=0, max=45, step=5, value=15,
        description='Slope Threshold (°):'
    )
)
def interactive_topographic_analysis(analysis_type, slope_threshold):
    
    if analysis_type == 'Slope Distribution':
        # Slope distribution analysis
        valid_slope = derivatives['slope'][~np.isnan(derivatives['slope'])]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Histogram
        ax1.hist(valid_slope, bins=50, alpha=0.7, color='orangered', edgecolor='black')
        ax1.axvline(slope_threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {slope_threshold}°')
        ax1.set_xlabel('Slope (degrees)')
        ax1.set_ylabel('Frequency')
        ax1.set_title('Slope Distribution')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Slope classes
        slope_classes = np.digitize(derivatives['slope'], [0, 5, 15, 30, 45, 90])
        slope_labels = ['Flat (0-5°)', 'Gentle (5-15°)', 'Moderate (15-30°)', 'Steep (30-45°)', 'Very Steep (45°+)']
        
        im = ax2.imshow(slope_classes, extent=extent, cmap='Reds', origin='upper')
        ax2.set_title('Slope Classification')
        ax2.set_xlabel('Longitude')
        ax2.set_ylabel('Latitude')
        
        # Calculate percentages
        unique, counts = np.unique(slope_classes[~np.isnan(slope_classes)], return_counts=True)
        percentages = counts / counts.sum() * 100
        
        print(f"📊 Slope Classification Results:")
        for i, (cls, pct) in enumerate(zip(unique, percentages)):
            if cls < len(slope_labels):
                print(f"   {slope_labels[int(cls)-1]:15}: {pct:5.1f}%")
        
        print(f"\n💡 Terrain above {slope_threshold}°: {np.sum(valid_slope > slope_threshold) / len(valid_slope) * 100:.1f}%")
    
    elif analysis_type == 'Aspect Analysis':
        # Aspect analysis with rose diagram
        valid_aspect = derivatives['aspect'][~np.isnan(derivatives['aspect'])]
        
        # Create rose diagram data
        aspect_bins = np.arange(0, 361, 45)  # 8 directions
        aspect_counts, _ = np.histogram(valid_aspect, bins=aspect_bins)
        
        # Convert to radians for polar plot
        theta = np.deg2rad(aspect_bins[:-1] + 22.5)  # Center of each bin
        
        fig = go.Figure()
        
        fig.add_trace(go.Scatterpolar(
            r=aspect_counts,
            theta=np.rad2deg(theta),
            fill='toself',
            name='Aspect Distribution'
        ))
        
        fig.update_layout(
            polar=dict(
                radialaxis=dict(visible=True, range=[0, max(aspect_counts)]),
                angularaxis=dict(
                    tickmode='array',
                    tickvals=[0, 45, 90, 135, 180, 225, 270, 315],
                    ticktext=['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
                )
            ),
            title='Aspect Rose Diagram',
            width=600, height=600
        )
        
        fig.show()
        
        # Aspect statistics
        directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
        print(f"📊 Aspect Distribution:")
        for i, (direction, count) in enumerate(zip(directions, aspect_counts)):
            percentage = count / aspect_counts.sum() * 100
            print(f"   {direction:3}: {percentage:5.1f}% ({count:4d} pixels)")
    
    elif analysis_type == 'Elevation Profiles':
        # Extract elevation profiles
        height, width = sample_elevation.shape
        
        # Horizontal profile (middle row)
        h_profile = sample_elevation[height//2, :]
        h_distance = np.linspace(extent[0], extent[1], width)
        
        # Vertical profile (middle column)
        v_profile = sample_elevation[:, width//2]
        v_distance = np.linspace(extent[3], extent[2], height)  # Top to bottom
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
        
        # Horizontal profile
        ax1.plot(h_distance, h_profile, 'b-', linewidth=2, label='W-E Profile')
        ax1.fill_between(h_distance, h_profile, alpha=0.3)
        ax1.set_xlabel('Longitude')
        ax1.set_ylabel('Elevation (m)')
        ax1.set_title('West-East Elevation Profile (Middle Row)')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
        
        # Vertical profile
        ax2.plot(v_distance, v_profile, 'r-', linewidth=2, label='N-S Profile')
        ax2.fill_between(v_distance, v_profile, alpha=0.3, color='red')
        ax2.set_xlabel('Latitude')
        ax2.set_ylabel('Elevation (m)')
        ax2.set_title('North-South Elevation Profile (Middle Column)')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
        
        # Profile statistics
        h_relief = np.nanmax(h_profile) - np.nanmin(h_profile)
        v_relief = np.nanmax(v_profile) - np.nanmin(v_profile)
        
        print(f"📏 Profile Analysis:")
        print(f"   W-E Relief: {h_relief:.1f} m")
        print(f"   N-S Relief: {v_relief:.1f} m")
        print(f"   Dominant trend: {'W-E' if h_relief > v_relief else 'N-S'}")
    
    elif analysis_type == 'Terrain Classification':
        # Classify terrain based on slope and curvature
        slope = derivatives['slope']
        plan_curv = derivatives['plan_curvature']
        profile_curv = derivatives['profile_curvature']
        
        # Define thresholds
        slope_thresh = 15  # degrees
        curv_thresh = 0.01  # curvature units
        
        # Classify terrain
        terrain_class = np.zeros_like(slope)
        
        # Flat areas
        terrain_class[slope < 5] = 1
        
        # Ridges (steep + convex profile)
        terrain_class[(slope >= slope_thresh) & (profile_curv > curv_thresh)] = 2
        
        # Valleys (convergent + concave profile)
        terrain_class[(plan_curv < -curv_thresh) & (profile_curv < -curv_thresh)] = 3
        
        # Slopes (moderate slope, low curvature)
        terrain_class[(slope >= 5) & (slope < slope_thresh) & 
                     (np.abs(plan_curv) <= curv_thresh) & 
                     (np.abs(profile_curv) <= curv_thresh)] = 4
        
        # Steep slopes
        terrain_class[(slope >= slope_thresh) & (terrain_class == 0)] = 5
        
        # Create custom colormap
        colors = ['white', 'lightgreen', 'brown', 'blue', 'orange', 'red']
        labels = ['Unclassified', 'Flat', 'Ridge', 'Valley', 'Gentle Slope', 'Steep Slope']
        
        fig, ax = plt.subplots(figsize=(12, 8))
        im = ax.imshow(terrain_class, extent=extent, origin='upper', cmap='Set1')
        ax.set_title('Geomorphological Terrain Classification')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        
        plt.show()
        
        # Calculate class percentages
        unique, counts = np.unique(terrain_class[~np.isnan(terrain_class)], return_counts=True)
        percentages = counts / counts.sum() * 100
        
        print(f"🗺️  Terrain Classification Results:")
        for cls, pct, count in zip(unique, percentages, counts):
            if int(cls) < len(labels):
                print(f"   {labels[int(cls)]:15}: {pct:5.1f}% ({count:4d} pixels)")
    
    plt.tight_layout()
    plt.show()

## 6. DEM Quality Assessment

### 6.1 Data Quality Metrics

In [ ]:
def assess_dem_quality(elevation_array, derivatives_dict):
    """
    Assess DEM quality using multiple metrics
    
    Parameters:
    elevation_array: 2D elevation array
    derivatives_dict: Dictionary with slope, aspect, etc.
    
    Returns:
    Dictionary with quality metrics
    """
    valid_elevation = elevation_array[~np.isnan(elevation_array)]
    
    quality_metrics = {
        'data_completeness': len(valid_elevation) / elevation_array.size * 100,
        'elevation_range': np.ptp(valid_elevation),
        'elevation_std': np.std(valid_elevation),
        'mean_slope': np.nanmean(derivatives_dict['slope']),
        'slope_std': np.nanstd(derivatives_dict['slope']),
        'terrain_roughness': np.nanstd(derivatives_dict['slope']),
        'aspect_uniformity': calculate_aspect_uniformity(derivatives_dict['aspect']),
        'noise_level': estimate_noise_level(elevation_array)
    }
    
    return quality_metrics

def calculate_aspect_uniformity(aspect_array):
    """
    Calculate aspect uniformity (0 = uniform, 1 = highly variable)
    """
    valid_aspect = aspect_array[~np.isnan(aspect_array)]
    
    # Convert to unit vectors
    aspect_rad = np.deg2rad(valid_aspect)
    x_components = np.cos(aspect_rad)
    y_components = np.sin(aspect_rad)
    
    # Calculate vector strength (opposite of uniformity)
    mean_x = np.mean(x_components)
    mean_y = np.mean(y_components)
    vector_strength = np.sqrt(mean_x**2 + mean_y**2)
    
    return 1 - vector_strength  # Convert to uniformity measure

def estimate_noise_level(elevation_array):
    """
    Estimate noise level using high-frequency content
    """
    from scipy import ndimage
    
    # Apply Laplacian filter to detect noise
    laplacian = ndimage.laplace(elevation_array)
    noise_estimate = np.nanstd(laplacian)
    
    return noise_estimate

# Assess quality of sample DEM
print("🔍 Assessing DEM Quality...")
quality_metrics = assess_dem_quality(sample_elevation, derivatives)

# Create quality report visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Data completeness
completeness = quality_metrics['data_completeness']
axes[0,0].pie([completeness, 100-completeness], labels=['Valid Data', 'No Data'], 
              colors=['green', 'red'], autopct='%1.1f%%')
axes[0,0].set_title(f'Data Completeness\n{completeness:.1f}% valid pixels')

# Elevation distribution
valid_elev = sample_elevation[~np.isnan(sample_elevation)]
axes[0,1].hist(valid_elev, bins=50, alpha=0.7, color='brown', edgecolor='black')
axes[0,1].set_xlabel('Elevation (m)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_title(f'Elevation Distribution\nRange: {quality_metrics["elevation_range"]:.0f} m')
axes[0,1].grid(True, alpha=0.3)

# Slope distribution
valid_slope = derivatives['slope'][~np.isnan(derivatives['slope'])]
axes[1,0].hist(valid_slope, bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[1,0].set_xlabel('Slope (degrees)')
axes[1,0].set_ylabel('Frequency')
axes[1,0].set_title(f'Slope Distribution\nMean: {quality_metrics["mean_slope"]:.1f}°')
axes[1,0].grid(True, alpha=0.3)

# Quality metrics summary
axes[1,1].axis('off')
quality_text = f"""
DEM Quality Assessment
{'='*25}

Data Completeness: {quality_metrics['data_completeness']:.1f}%
Elevation Range: {quality_metrics['elevation_range']:.1f} m
Elevation Std Dev: {quality_metrics['elevation_std']:.1f} m
Mean Slope: {quality_metrics['mean_slope']:.1f}°
Terrain Roughness: {quality_metrics['terrain_roughness']:.2f}
Aspect Uniformity: {quality_metrics['aspect_uniformity']:.3f}
Noise Level: {quality_metrics['noise_level']:.2f}

Quality Rating:
"""

# Simple quality rating
if quality_metrics['data_completeness'] > 95:
    if quality_metrics['noise_level'] < 1.0:
        rating = "Excellent ⭐⭐⭐⭐⭐"
    else:
        rating = "Good ⭐⭐⭐⭐"
elif quality_metrics['data_completeness'] > 90:
    rating = "Fair ⭐⭐⭐"
else:
    rating = "Poor ⭐⭐"

quality_text += rating

axes[1,1].text(0.05, 0.95, quality_text, transform=axes[1,1].transAxes,
               fontsize=11, verticalalignment='top', fontfamily='monospace',
               bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"\n📋 DEM Quality Summary:")
print(f"   Overall Rating: {rating}")
print(f"   Primary Strengths: {'High data completeness' if quality_metrics['data_completeness'] > 95 else 'Adequate coverage'}")
print(f"   Terrain Complexity: {'High' if quality_metrics['terrain_roughness'] > 10 else 'Moderate' if quality_metrics['terrain_roughness'] > 5 else 'Low'}")
print(f"   Suitable for EEMT: {'Yes' if quality_metrics['data_completeness'] > 90 else 'With limitations'}")

# Recommendations
print(f"\n💡 Recommendations:")
if quality_metrics['data_completeness'] < 95:
    print(f"   • Consider gap-filling for missing data")
if quality_metrics['noise_level'] > 2.0:
    print(f"   • Apply smoothing filter to reduce noise")
if quality_metrics['elevation_range'] < 100:
    print(f"   • Low relief area - topographic effects may be minimal")
if quality_metrics['aspect_uniformity'] > 0.8:
    print(f"   • High aspect variability indicates complex terrain")
    
print(f"   • DEM suitable for solar radiation modeling: {rating != 'Poor ⭐⭐'}")

## 7. Coordinate Reference Systems

### 7.1 CRS Analysis and Reprojection

In [ ]:
def analyze_dem_crs(dem_path):
    """
    Analyze coordinate reference system of DEM
    """
    with rasterio.open(dem_path) as src:
        crs_info = {
            'crs': src.crs,
            'crs_string': str(src.crs),
            'bounds': src.bounds,
            'transform': src.transform,
            'pixel_size': (src.transform[0], -src.transform[4]),
            'units': src.crs.linear_units if src.crs else 'unknown'
        }
        
        # Check if geographic or projected
        crs_info['is_geographic'] = src.crs.is_geographic if src.crs else None
        crs_info['is_projected'] = src.crs.is_projected if src.crs else None
        
    return crs_info

def recommend_projection(bounds, center_lat, center_lon):
    """
    Recommend appropriate projection for study area
    """
    recommendations = []
    
    # UTM zone calculation
    utm_zone = int((center_lon + 180) / 6) + 1
    hemisphere = 'N' if center_lat >= 0 else 'S'
    epsg_utm = 32600 + utm_zone if hemisphere == 'N' else 32700 + utm_zone
    
    recommendations.append({
        'name': f'UTM Zone {utm_zone}{hemisphere}',
        'epsg': epsg_utm,
        'reason': 'Best for local/regional analysis with accurate distances',
        'pros': 'Conformal, accurate distances and areas',
        'cons': 'Limited to 6° longitude bands'
    })
    
    # Equal Area projections for large areas
    if abs(bounds.right - bounds.left) > 2 or abs(bounds.top - bounds.bottom) > 2:
        recommendations.append({
            'name': 'Lambert Azimuthal Equal Area',
            'epsg': None,
            'reason': 'Best for large area analysis requiring accurate areas',
            'pros': 'Equal area, good for continental scale',
            'cons': 'Shape distortion increases with distance from center'
        })
    
    # Web Mercator for visualization
    recommendations.append({
        'name': 'Web Mercator',
        'epsg': 3857,
        'reason': 'Good for web mapping and visualization',
        'pros': 'Compatible with web maps, familiar',
        'cons': 'Area distortion, not suitable for analysis at high latitudes'
    })
    
    return recommendations

# Analyze sample DEM CRS
print("🗺️ Analyzing Coordinate Reference System...")
crs_info = analyze_dem_crs(sample_dem_path)

print(f"\n📍 Current CRS Information:")
print(f"   CRS: {crs_info['crs_string']}")
print(f"   Type: {'Geographic' if crs_info['is_geographic'] else 'Projected' if crs_info['is_projected'] else 'Unknown'}")
print(f"   Units: {crs_info['units']}")
print(f"   Bounds: W={crs_info['bounds'].left:.3f}, S={crs_info['bounds'].bottom:.3f}, E={crs_info['bounds'].right:.3f}, N={crs_info['bounds'].top:.3f}")
print(f"   Pixel Size: {crs_info['pixel_size'][0]:.6f} x {crs_info['pixel_size'][1]:.6f} {crs_info['units']}")

# Get center coordinates
center_lat = (crs_info['bounds'].bottom + crs_info['bounds'].top) / 2
center_lon = (crs_info['bounds'].left + crs_info['bounds'].right) / 2

print(f"   Center: {center_lat:.3f}°N, {center_lon:.3f}°W")

# Get projection recommendations
recommendations = recommend_projection(crs_info['bounds'], center_lat, center_lon)

print(f"\n🎯 Recommended Projections:")
for i, rec in enumerate(recommendations, 1):
    print(f"\n   {i}. {rec['name']}")
    if rec['epsg']:
        print(f"      EPSG: {rec['epsg']}")
    print(f"      Reason: {rec['reason']}")
    print(f"      Pros: {rec['pros']}")
    print(f"      Cons: {rec['cons']}")

# Demonstrate reprojection to UTM
print(f"\n🔄 Demonstration: Reprojecting to UTM...")

utm_zone = int((center_lon + 180) / 6) + 1
hemisphere = 'N' if center_lat >= 0 else 'S'
epsg_utm = 32600 + utm_zone if hemisphere == 'N' else 32700 + utm_zone
target_crs = f'EPSG:{epsg_utm}'

with rasterio.open(sample_dem_path) as src:
    # Calculate transform for reprojection
    transform, width, height = calculate_default_transform(
        src.crs, target_crs, src.width, src.height, *src.bounds)
    
    # Create profile for reprojected raster
    kwargs = src.profile.copy()
    kwargs.update({
        'crs': target_crs,
        'transform': transform,
        'width': width,
        'height': height
    })
    
    # Calculate new pixel size
    utm_pixel_size = (transform[0], -transform[4])
    
    print(f"   Original CRS: {src.crs}")
    print(f"   Target CRS: {target_crs} (UTM Zone {utm_zone}{hemisphere})")
    print(f"   Original pixel size: {crs_info['pixel_size'][0]:.6f}° x {crs_info['pixel_size'][1]:.6f}°")
    print(f"   UTM pixel size: {utm_pixel_size[0]:.1f}m x {utm_pixel_size[1]:.1f}m")
    print(f"   Original dimensions: {src.width} x {src.height}")
    print(f"   UTM dimensions: {width} x {height}")

print(f"\n💡 For EEMT Calculations:")
print(f"   • Geographic (WGS84) coordinates work well for DAYMET climate data alignment")
print(f"   • UTM projections provide accurate areas and distances for local analysis")
print(f"   • Choose projection based on your study area size and analysis requirements")
print(f"   • GRASS GIS can handle reprojection automatically during solar radiation calculations")

## 8. Summary and Next Steps

### 8.1 Key Takeaways

In [ ]:
# Create summary visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Data Sources Comparison', 'Resolution vs Coverage Trade-off',
                   'Quality Assessment Framework', 'Processing Workflow'],
    specs=[[{"type": "table"}, {"type": "scatter"}],
           [{"type": "bar"}, {"type": "scatter"}]]
)

# Data sources summary
summary_data = [
    ['USGS 3DEP', 'US Only', '1-30m', 'Excellent', 'Yes'],
    ['SRTM GL1', 'Global', '30m', 'Good', 'Yes'],
    ['ALOS World', 'Global', '30m', 'Very Good', 'Yes'],
    ['Copernicus', 'Global', '30-90m', 'Very Good', 'Limited'],
    ['FABDEM', 'Global', '30m', 'Good*', 'Via GEE']
]

fig.add_trace(go.Table(
    header=dict(values=['Source', 'Coverage', 'Resolution', 'Quality', 'API'],
               fill_color='lightblue'),
    cells=dict(values=list(zip(*summary_data)),
              fill_color='white')
), row=1, col=1)

# Resolution vs coverage scatter
sources = ['3DEP-1m', '3DEP-10m', '3DEP-30m', 'SRTM-30m', 'ALOS-30m', 'COP-30m', 'COP-90m']
resolutions = [1, 10, 30, 30, 30, 30, 90]
coverage_scores = [1, 1, 1, 10, 10, 10, 10]  # 1=US, 10=Global
quality_scores = [10, 9, 8, 6, 8, 8, 7]  # Relative quality

fig.add_trace(go.Scatter(
    x=resolutions, y=coverage_scores,
    mode='markers+text',
    marker=dict(size=quality_scores, sizemode='area', sizeref=0.5, color=quality_scores,
               colorscale='Viridis'),
    text=sources, textposition='top center',
    name='Data Sources'
), row=1, col=2)

fig.update_xaxes(title_text="Resolution (m)", type="log", row=1, col=2)
fig.update_yaxes(title_text="Coverage (1=US, 10=Global)", row=1, col=2)

# Quality metrics
quality_categories = ['Completeness', 'Accuracy', 'Resolution', 'Coverage', 'API Access']
current_scores = [90, 85, 70, 95, 80]  # Example scores

fig.add_trace(go.Bar(
    x=quality_categories, y=current_scores,
    marker_color=['green' if score >= 80 else 'orange' if score >= 60 else 'red' 
                  for score in current_scores],
    name='Quality Scores'
), row=2, col=1)

fig.update_yaxes(title_text="Score (0-100)", row=2, col=1)

# Processing workflow
workflow_steps = ['Download', 'Validate', 'Derivatives', 'Quality Check', 'Ready for EEMT']
completion = [100, 100, 100, 100, 100]  # All steps completed

fig.add_trace(go.Scatter(
    x=workflow_steps, y=completion,
    mode='lines+markers',
    line=dict(width=4, color='green'),
    marker=dict(size=10, color='green'),
    name='Workflow Progress'
), row=2, col=2)

fig.update_yaxes(title_text="Completion (%)", row=2, col=2)

fig.update_layout(
    title='Elevation Data Analysis Summary',
    height=700, width=1200,
    showlegend=False
)

fig.show()

print("🎯 Key Learning Outcomes Achieved:")
print("   ✅ Understand multiple public elevation data sources")
print("   ✅ Access and download DEM data programmatically")
print("   ✅ Validate data quality and assess fitness for use")
print("   ✅ Calculate topographic derivatives (slope, aspect, curvature)")
print("   ✅ Understand coordinate reference systems and projections")
print("   ✅ Visualize topographic data effectively")

print("\n💡 Best Practices Summary:")
print("   🎯 Choose data source based on study area and resolution requirements")
print("   🔍 Always validate downloaded data before analysis")
print("   📏 Consider coordinate reference system for accurate calculations")
print("   🗺️  Use topographic derivatives to understand terrain characteristics")
print("   ⚖️  Balance resolution, coverage, and computational requirements")

print("\n🚀 Next Steps:")
print("   → 02_climate_data.ipynb: Access DAYMET and other climate datasets")
print("   → ../03_grass_workflows/01_grass_setup.ipynb: GRASS GIS environment setup")
print("   → ../03_grass_workflows/02_solar_modeling.ipynb: Solar radiation calculations")
print("   → ../04_calculation_methods/: EEMT calculation implementations")

print("\n📚 Additional Resources:")
print("   • OpenTopography: https://opentopography.org/")
print("   • USGS 3DEP: https://www.usgs.gov/3d-elevation-program")
print("   • Copernicus DEM: https://spacedata.copernicus.eu/")
print("   • FABDEM: https://data.bris.ac.uk/data/dataset/25wfy0f9ukoge2gs7a5mqpq2j7")

## 9. Exercises and Extensions

### 9.1 Practice Exercises

1. **Local Area Analysis**:
   - Download elevation data for your local area
   - Calculate topographic derivatives
   - Identify dominant terrain features

2. **Multi-source Comparison**:
   - Download the same area from different sources (SRTM vs ALOS)
   - Compare elevation values and topographic derivatives
   - Analyze differences and their potential impact on EEMT

3. **Resolution Analysis**:
   - Resample high-resolution DEM to coarser resolutions
   - Compare slope and aspect calculations at different resolutions
   - Determine optimal resolution for your study objectives

### 9.2 Advanced Projects

1. **Terrain Classification**:
   - Implement advanced landform classification algorithms
   - Use machine learning to classify terrain types
   - Validate classifications against field data or imagery

2. **Error Analysis**:
   - Implement uncertainty propagation for topographic derivatives
   - Analyze how DEM errors affect EEMT calculations
   - Develop quality control procedures for your analysis

3. **Scale Effects**:
   - Investigate how terrain analysis results change with scale
   - Implement scale-dependent terrain analysis
   - Develop guidelines for appropriate scale selection

### 9.3 Real-World Applications

1. **Hazard Assessment**:
   - Use slope analysis for landslide susceptibility mapping
   - Identify flood-prone areas using flow accumulation
   - Assess erosion potential from topographic metrics

2. **Renewable Energy**:
   - Site wind turbines using terrain exposure analysis
   - Assess solar potential with topographic solar modeling
   - Optimize placement using viewshed analysis

3. **Ecosystem Modeling**:
   - Use topographic variables to predict species distributions
   - Model ecosystem boundaries using terrain analysis
   - Assess habitat connectivity across landscapes

---

**Remember**: High-quality elevation data is fundamental to accurate EEMT calculations. Take time to understand your data sources, validate quality, and choose appropriate processing methods for your specific application.